In [ ]:
import os
import pandas as pd
import numpy as np

csv_folder = "/content/drive/MyDrive/dataset/DDOS/CSV-01-12 (1)/01-12"
print(os.listdir(csv_folder))


In [ ]:
csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

print("Step 1: CSV files detected")
for f in csv_files:
    print("  -", f)

In [ ]:
output_folder = "/content/drive/MyDrive/dataset/DDOS/processed_chunks"
os.makedirs(output_folder, exist_ok=True)
label_col = "Label"
CHUNK_SIZE = 100_000

In [ ]:
for file_no, file_name in enumerate(csv_files, start=1):
    file_path = os.path.join(csv_folder, file_name)
    base_name = file_name.replace(".csv", "")

    print(f"\n==============================")
    print(f"Step 2.{file_no}: Processing file → {file_name}")
    print("==============================")


In [ ]:
# Count total rows (cheap, no full load)
total_rows = sum(1 for _ in open(file_path)) - 1  # minus header
total_chunks = (total_rows // CHUNK_SIZE) + 1

print(f"Total rows in {file_name}: {total_rows}")
print(f"Total chunks expected: {total_chunks}")


In [ ]:
for chunk_idx, chunk in enumerate(
    pd.read_csv(file_path, chunksize=CHUNK_SIZE)
):
    print(f"\n  ▶ File: {file_name}")
    print(f"    Chunk: {chunk_idx + 1} / {total_chunks}")
    print(f"    Shape before processing: {chunk.shape}")


In [ ]:
chunk = process_chunk_keep_all(chunk, label_col)

In [ ]:
        # Process chunk
chunk = process_chunk_keep_all(chunk, label_col)

        # Chunk size details
rows, cols = chunk.shape
memory_mb = chunk.memory_usage(deep=True).sum() / (1024 ** 2)

print(f"    Shape AFTER processing: {rows, cols}")
print(f"    Rows in this chunk: {rows}")
print(f"    Memory usage: {memory_mb:.2f} MB")


In [ ]:
print("Columns in chunk:")
print(chunk.columns.tolist())


In [ ]:
chunk.columns = chunk.columns.str.strip()


In [ ]:
print("    Label distribution:")
print(chunk[label_col].value_counts())


In [ ]:
print(f"    Shape after processing: {chunk.shape}")
print(f"    Label distribution:")
print(chunk[label_col].value_counts())


In [ ]:
import os
import pandas as pd
import numpy as np

# ---------- CONFIG ----------
csv_folder = "/content/drive/MyDrive/dataset/DDOS/CSV-01-12 (1)/01-12"
output_folder = "/content/drive/MyDrive/dataset/DDOS/processed_chunks"
os.makedirs(output_folder, exist_ok=True)

CHUNK_SIZE = 100_000

# -----------------------------

# ---------- HELPER FUNCTION ----------
def process_chunk_keep_all(chunk):
    # 1. Strip column names
    chunk.columns = chunk.columns.str.strip()

    # 2. Detect label column dynamically
    label_col = None
    for c in chunk.columns:
        if "label" in c.lower():
            label_col = c
            break
    if label_col is None:
        raise ValueError("Label column not found in this chunk")

    # 3. Replace ±inf with NaN
    chunk.replace([np.inf, -np.inf], np.nan, inplace=True)

    # 4. Encode label (BENIGN=0, ATTACK=1)
    chunk[label_col] = (chunk[label_col] != "BENIGN").astype("int8")

    # 5. Convert object columns (except label) to numeric
    for col in chunk.select_dtypes(include=['object']).columns:
        if col != label_col:
            chunk[col] = pd.to_numeric(chunk[col], errors='coerce')

    # 6. Drop columns that are entirely NaN
    chunk.dropna(axis=1, how='all', inplace=True)

    # 7. Drop rows only if features are NaN (exclude label)
    feature_cols = [c for c in chunk.columns if c != label_col]
    chunk.dropna(subset=feature_cols, inplace=True)

    # 8. Downcast numeric types for memory efficiency
    for col in chunk.select_dtypes(include=['float64']):
        chunk[col] = chunk[col].astype('float32')
    for col in chunk.select_dtypes(include=['int64']):
        chunk[col] = chunk[col].astype('int32')

    return chunk, label_col

# -----------------------------

# ---------- MAIN PROCESS ----------
csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]
print("CSV files detected:")
for f in csv_files:
    print(" -", f)

for file_no, file_name in enumerate(csv_files, start=1):
    file_path = os.path.join(csv_folder, file_name)
    base_name = file_name.replace(".csv", "")

    print(f"\n==============================")
    print(f"Processing file {file_no}/{len(csv_files)} → {file_name}")
    print("==============================")

    # Count total rows to estimate chunks
    total_rows = sum(1 for _ in open(file_path)) - 1
    total_chunks = (total_rows // CHUNK_SIZE) + 1
    print(f"Total rows: {total_rows}")
    print(f"Expected chunks: {total_chunks}")

    for chunk_idx, chunk in enumerate(pd.read_csv(file_path, chunksize=CHUNK_SIZE, low_memory=False)):
        print(f"\n  ▶ Chunk {chunk_idx + 1}/{total_chunks}")
        print(f"    Shape BEFORE: {chunk.shape}")


        # Process chunk safely
        chunk, label_col_in_chunk = process_chunk_keep_all(chunk)

        # ================= SAVE CLEANED CHUNK =================
        output_folder,
        out_file = os.path.join(
        f"{base_name}_chunk_{chunk_idx}.parquet"
          )

        # --------- SKIP IF ALREADY SAVED ---------
        if os.path.exists(out_file):
         print(f"    Skipping (already exists): {out_file}")
        else:
            chunk.to_parquet(out_file, index=False)
            print(f"    Saved cleaned chunk → {out_file}")
        # =====================================================

        rows, cols = chunk.shape
        memory_mb = chunk.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"    Shape AFTER: ({rows}, {cols})")
        print(f"    Memory usage: {memory_mb:.2f} MB")
        print(f"    Label distribution:")
        print(chunk[label_col_in_chunk].value_counts())

**VARIANCE THRESHOLD**

In [ ]:
import os
import pandas as pd
import numpy as np

CLEANED_DIR = "/content/drive/MyDrive/dataset/DDOS/processed_chunks"
VAR_SAMPLE_CHUNKS = 5
VAR_THRESHOLD = 0.0
SAVE_PATH = "/content/drive/MyDrive/dataset/DDOS/variance_features.npy"

files = sorted([f for f in os.listdir(CLEANED_DIR) if f.endswith(".parquet")])
sample_dfs = []

for i, file in enumerate(files[:VAR_SAMPLE_CHUNKS]):
    df = pd.read_parquet(os.path.join(CLEANED_DIR, file))

    label_col = [c for c in df.columns if "label" in c.lower()][0]
    X = df.drop(columns=[label_col])

    sample_dfs.append(X)
    print(f"Loaded sample {i+1}: {file}")

X_sample = pd.concat(sample_dfs, axis=0)
variances = X_sample.var()

low_var = variances[variances <= VAR_THRESHOLD].index.tolist()
kept = variances[variances > VAR_THRESHOLD].index.tolist()

print("\nZero-variance features:")
for f in low_var:
    print(" -", f)

print(f"\nFeatures before: {X_sample.shape[1]}")
print(f"Features after : {len(kept)}")

np.save(SAVE_PATH, kept)
print("\nSaved variance features →", SAVE_PATH)


In [ ]:
import numpy as np
import pandas as pd

# Load selected features
kept_features = np.load(
    "/content/drive/MyDrive/dataset/DDOS/variance_features.npy",
    allow_pickle=True
)

# Load one large sample again
CLEANED_DIR = "/content/drive/MyDrive/dataset/DDOS/processed_chunks"
files = sorted([f for f in os.listdir(CLEANED_DIR) if f.endswith(".parquet")])

dfs = []
for f in files[:5]:
    df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
    label_col = [c for c in df.columns if "label" in c.lower()][0]
    dfs.append(df[kept_features])

X = pd.concat(dfs)

# Compute variances
variances = X.var().sort_values()

print("🔽 Lowest variance features:")
print(variances.head(35))

print("\n🔼 Highest variance features:")
print(variances.tail(35))


In [ ]:
import os
import numpy as np
import pandas as pd

# ---------------- CONFIG ----------------
CLEANED_DIR = "/content/drive/MyDrive/dataset/DDOS/processed_chunks"
VAR_FEATURES_PATH = "/content/drive/MyDrive/dataset/DDOS/variance_features.npy"
CORR_THRESHOLD = 0.9
SAMPLE_CHUNKS = 5
SAVE_PATH = "/content/drive/MyDrive/dataset/DDOS/corr_features.npy"
# ---------------------------------------

# Load variance-selected features
features = np.load(VAR_FEATURES_PATH, allow_pickle=True)

# Load sample data
dfs = []
files = sorted([f for f in os.listdir(CLEANED_DIR) if f.endswith(".parquet")])

for f in files[:SAMPLE_CHUNKS]:
    df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
    label_col = [c for c in df.columns if "label" in c.lower()][0]

    # Keep only variance-selected features
    df = df[features]

    # Drop obvious junk if present
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])

    dfs.append(df)

X = pd.concat(dfs)
print("Sample shape used for correlation:", X.shape)

# ---------------- CORRELATION ----------------
corr_matrix = X.corr().abs()

upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

to_drop = [
    column for column in upper_triangle.columns
    if any(upper_triangle[column] >= CORR_THRESHOLD)
]

kept_features = [f for f in X.columns if f not in to_drop]

# ---------------- OUTPUT ----------------
print("\nCorrelation threshold:", CORR_THRESHOLD)
print("Features before correlation:", X.shape[1])
print("Highly correlated features removed:", len(to_drop))
print("Features after correlation:", len(kept_features))

print("\nSample removed features:")
for f in to_drop[:15]:
    print(" -", f)

# Save final correlated-filtered features
np.save(SAVE_PATH, kept_features)
print("\nSaved correlation-filtered features →", SAVE_PATH)


In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

# ---------------- CONFIG ----------------
CLEANED_DIR = "/content/drive/MyDrive/dataset/DDOS/processed_chunks"
CORR_FEATURES_PATH = "/content/drive/MyDrive/dataset/DDOS/corr_features.npy"

SAMPLE_CHUNKS = 5
K_FEATURES = 25   # final target features

SAVE_ANOVA = "/content/drive/MyDrive/dataset/DDOS/anova_features.npy"
SAVE_MI = "/content/drive/MyDrive/dataset/DDOS/mi_features.npy"
SAVE_FINAL = "/content/drive/MyDrive/dataset/DDOS/final_features.npy"
# --------------------------------------

# Load correlation-filtered features
features = np.load(CORR_FEATURES_PATH, allow_pickle=True)

# Load sample data
dfs = []
files = sorted([f for f in os.listdir(CLEANED_DIR) if f.endswith(".parquet")])

for f in files[:SAMPLE_CHUNKS]:
    df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
    label_col = [c for c in df.columns if "label" in c.lower()][0]

    df = df[features.tolist() + [label_col]]

    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])

    dfs.append(df)

data = pd.concat(dfs)

X = data.drop(columns=[label_col])
y = data[label_col]

print("Sample used for ANOVA + MI:", X.shape)

# ---------------- ANOVA ----------------
anova_selector = SelectKBest(score_func=f_classif, k=K_FEATURES)
anova_selector.fit(X, y)

anova_scores = pd.Series(
    anova_selector.scores_,
    index=X.columns
).sort_values(ascending=False)

anova_selected = anova_scores.head(K_FEATURES).index.tolist()

np.save(SAVE_ANOVA, anova_selected)

print("\nTop ANOVA features:")
for f in anova_selected:
    print(" -", f)

# ---------------- MUTUAL INFORMATION ----------------
mi_scores = mutual_info_classif(
    X, y,
    discrete_features=False,
    random_state=42
)

mi_scores = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
mi_selected = mi_scores.head(K_FEATURES).index.tolist()

np.save(SAVE_MI, mi_selected)

print("\nTop MI features:")
for f in mi_selected:
    print(" -", f)

# ---------------- COMBINE (INTERSECTION) ----------------
final_features = sorted(list(set(anova_selected) & set(mi_selected)))

np.save(SAVE_FINAL, final_features)

print("\nFinal selected features (ANOVA ∩ MI):")
for f in final_features:
    print(" -", f)

print(f"\nFinal feature count: {len(final_features)}")
print("Saved final features →", SAVE_FINAL)


**CELL 1 — APPLY FINAL FEATURES TO ALL CHUNKS**

In [ ]:
import os
import pandas as pd
import numpy as np

# ---------------- CONFIG ----------------
CLEANED_DIR = "/content/drive/MyDrive/dataset/DDOS/processed_chunks"
FINAL_FEATURES_PATH = "/content/drive/MyDrive/dataset/DDOS/final_features.npy"

X_list = []
y_list = []

final_features = np.load(FINAL_FEATURES_PATH, allow_pickle=True)

# List all parquet chunks
files = sorted([f for f in os.listdir(CLEANED_DIR) if f.endswith(".parquet")])

for f in files:
    df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
    label_col = [c for c in df.columns if "label" in c.lower()][0]

    # Keep only final features + label
    cols_to_keep = final_features.tolist() + [label_col]
    df = df[cols_to_keep]

    # Append to list
    X_list.append(df.drop(columns=[label_col]))
    y_list.append(df[label_col])

# Concatenate all chunks
X = pd.concat(X_list, ignore_index=True)
y = pd.concat(y_list, ignore_index=True)

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Label distribution:")
print(y.value_counts(normalize=True))


RAM CRASHED FOR ANN

In [ ]:
# ============================================================
# ANN (MLP) MODEL TRAINING – CICDDoS2019
# ============================================================

import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# ---------------- CONFIG ----------------
CLEANED_DIR = "/content/drive/MyDrive/dataset/DDOS/processed_chunks"
FINAL_FEATURES_PATH = "/content/drive/MyDrive/dataset/DDOS/final_features.npy"

EPOCHS = 10
BATCH_SIZE = 1024
# ---------------------------------------

# Load final selected features
final_features = np.load(FINAL_FEATURES_PATH, allow_pickle=True).tolist()

# List parquet chunks
files = sorted([f for f in os.listdir(CLEANED_DIR) if f.endswith(".parquet")])

# -------- SPLIT BY CHUNKS (NO LEAKAGE) --------
n_total = len(files)
train_files = files[:int(0.7 * n_total)]
val_files   = files[int(0.7 * n_total):int(0.85 * n_total)]
test_files  = files[int(0.85 * n_total):]

def load_chunks(file_list):
    X_list, y_list = [], []

    for f in file_list:
        df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
        label_col = [c for c in df.columns if "label" in c.lower()][0]

        df = df[final_features + [label_col]]

        X_list.append(df.drop(columns=[label_col]))
        y_list.append(df[label_col])

    return pd.concat(X_list), pd.concat(y_list)

# Load datasets
X_train, y_train = load_chunks(train_files)
X_val,   y_val   = load_chunks(val_files)
X_test,  y_test  = load_chunks(test_files)

print("Train shape:", X_train.shape)
print("Val shape  :", X_val.shape)
print("Test shape :", X_test.shape)

# ---------------- NORMALIZATION ----------------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)
X_test  = scaler.transform(X_test)

# ---------------- CLASS WEIGHTS ----------------
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0, 1]),
    y=y_train
)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

# ---------------- BUILD ANN ----------------
def build_ann(input_dim):
    model = Sequential([
        Dense(128, activation="relu", input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.3),

        Dense(64, activation="relu"),
        BatchNormalization(),
        Dropout(0.3),

        Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

model = build_ann(X_train.shape[1])
model.summary()

# ---------------- TRAIN ----------------
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)

# ---------------- EVALUATION ----------------
y_prob = model.predict(X_test).ravel()
y_pred = (y_prob > 0.5).astype(int)

print("\nANN Classification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC:", roc_auc_score(y_test, y_prob))


In [ ]:
# ============================================================
# MEMORY-SAFE ANN TRAINING WITH SAMPLE WEIGHTS (CICDDoS2019)
# ============================================================

import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# ---------------- CONFIG ----------------
CLEANED_DIR = "/content/drive/MyDrive/dataset/DDOS/processed_chunks"
FINAL_FEATURES_PATH = "/content/drive/MyDrive/dataset/DDOS/final_features.npy"

EPOCHS = 1
BATCH_SIZE = 1024
# --------------------------------------

# ---------------- LOAD FEATURES ----------------
final_features = np.load(FINAL_FEATURES_PATH, allow_pickle=True).tolist()

files = sorted([f for f in os.listdir(CLEANED_DIR) if f.endswith(".parquet")])

# ---------------- SPLIT BY FILES ----------------
n_total = len(files)
train_files = files[:int(0.7 * n_total)]
val_files   = files[int(0.7 * n_total):int(0.85 * n_total)]
test_files  = files[int(0.85 * n_total):]

print("Train files:", len(train_files))
print("Val files  :", len(val_files))
print("Test files :", len(test_files))

# ---------------- COMPUTE CLASS WEIGHTS ----------------
y_all = []

for f in train_files[:10]:  # sample only (fast)
    df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
    label_col = [c for c in df.columns if "label" in c.lower()][0]
    y_all.append(df[label_col])

y_all = pd.concat(y_all)

cw = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0, 1]),
    y=y_all
)

class_weight_dict = {0: cw[0], 1: cw[1]}
print("Class weights:", class_weight_dict)

# ---------------- SCALER (FIT ON SMALL SAMPLE) ----------------
scaler = StandardScaler()

sample_df = pd.read_parquet(os.path.join(CLEANED_DIR, train_files[0]))
label_col = [c for c in sample_df.columns if "label" in c.lower()][0]
scaler.fit(sample_df[final_features])

# ---------------- DATA GENERATOR ----------------
def data_generator(file_list, batch_size):
    while True:
        for f in file_list:
            df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
            label_col = [c for c in df.columns if "label" in c.lower()][0]

            X = df[final_features].values
            y = df[label_col].values

            X = scaler.transform(X)

            sample_weights = np.array(
                [class_weight_dict[int(label)] for label in y]
            )

            for i in range(0, len(X), batch_size):
                yield (
                    X[i:i+batch_size],
                    y[i:i+batch_size],
                    sample_weights[i:i+batch_size]
                )

# ---------------- STEPS PER EPOCH ----------------
def count_rows(file_list):
    return sum(len(pd.read_parquet(os.path.join(CLEANED_DIR, f))) for f in file_list)

steps_per_epoch = count_rows(train_files) // BATCH_SIZE
val_steps = count_rows(val_files) // BATCH_SIZE

# ---------------- BUILD ANN ----------------
def build_ann(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),

        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.3),

        Dense(64, activation="relu"),
        BatchNormalization(),
        Dropout(0.3),

        Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

model = build_ann(len(final_features))
model.summary()

# ---------------- TRAIN ----------------
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    data_generator(train_files, BATCH_SIZE),
    validation_data=data_generator(val_files, BATCH_SIZE),
    steps_per_epoch=steps_per_epoch,
    validation_steps=val_steps,
    epochs=EPOCHS,
    callbacks=[early_stop],
    verbose=1
)

# ---------------- TEST EVALUATION ----------------
X_test_list, y_test_list = [], []

for f in test_files:
    df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
    label_col = [c for c in df.columns if "label" in c.lower()][0]

    X_test_list.append(df[final_features])
    y_test_list.append(df[label_col])

X_test = scaler.transform(pd.concat(X_test_list))
y_test = pd.concat(y_test_list)

y_prob = model.predict(X_test, batch_size=2048).ravel()
y_pred = (y_prob > 0.5).astype(int)

print("\nANN Classification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC:", roc_auc_score(y_test, y_prob))


Train files: 354
Val files  : 76
Test files : 76
Class weights: {0: np.float64(263.05674342105266), 1: np.float64(0.5009521751250513)}


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,521 (45.00 KB)

 Trainable params: 11,137 (43.50 KB)

...

33190/33190 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9571 - loss: 0.4759

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut

33190/33190 ━━━━━━━━━━━━━━━━━━━━ 493s 15ms/step - accuracy: 0.9571 - loss: 0.4759 - val_accuracy: 0.9990 - val_loss: 0.1221


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


3550/3550 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step

ANN Classification Report:
              precision    recall  f1-score   support

           0       0.02      1.00      0.03      4765
           1       1.00      0.96      0.98   7264104

    accuracy                           0.96   7268869
   macro avg       0.51      0.98      0.51   7268869
weighted avg       1.00      0.96      0.98   7268869

ROC-AUC: 0.9979971529459164


GREY wolf with correlation filtered feature dataset

GWO with ANN

In [ ]:
# ============================================================
# ANN TRAINING USING GWO-SELECTED FEATURES
# ============================================================

import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# ---------------- CONFIG ----------------
CLEANED_DIR = "/content/drive/MyDrive/dataset/DDOS/processed_chunks"
GWO_FEATURES_PATH = "/content/drive/MyDrive/dataset/DDOS/gwo_features.npy"

EPOCHS = 3
BATCH_SIZE = 1024
# --------------------------------------

gwo_features = np.load(GWO_FEATURES_PATH, allow_pickle=True).tolist()

files = sorted([f for f in os.listdir(CLEANED_DIR) if f.endswith(".parquet")])
n = len(files)

train_files = files[:int(0.7*n)]
val_files   = files[int(0.7*n):int(0.85*n)]
test_files  = files[int(0.85*n):]

# ---------------- SCALER ----------------
sample_df = pd.read_parquet(os.path.join(CLEANED_DIR, train_files[0]))
label_col = [c for c in sample_df.columns if "label" in c.lower()][0]

scaler = StandardScaler()
scaler.fit(sample_df[gwo_features])

# ---------------- GENERATOR ----------------
def generator(file_list):
    while True:
        for f in file_list:
            df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
            X = scaler.transform(df[gwo_features])
            y = df[label_col].values
            yield X, y

# ---------------- BUILD ANN ----------------
model = Sequential([
    Input(shape=(len(gwo_features),)),
    Dense(64, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=Adam(0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ---------------- TRAIN ----------------
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

model.fit(
    generator(train_files),
    validation_data=generator(val_files),
    steps_per_epoch=50,
    validation_steps=20,
    epochs=EPOCHS,
    callbacks=[early_stop],
    verbose=1
)

# ---------------- TEST ----------------
X_test, y_test = [], []

for f in test_files:
    df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
    X_test.append(df[gwo_features])
    y_test.append(df[label_col])

X_test = scaler.transform(pd.concat(X_test))
y_test = pd.concat(y_test)

y_prob = model.predict(X_test, batch_size=2048).ravel()
y_pred = (y_prob > 0.5).astype(int)

print("\nANN + GWO Results:")
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,561 (10.00 KB)

 Trainable params: 2,433 (9.50 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/3
50/50 ━━━━━━━━━━━━━━━━━━━━ 23s 429ms/step - accuracy: 0.9838 - loss: 0.6143 - val_accuracy: 0.9998 - val_loss: 0.2432
Epoch 2/3
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 348ms/step - accuracy: 0.9996 - loss: 0.1638 - val_accuracy: 0.9998 - val_loss: 0.0174
Epoch 3/3
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 407ms/step - accuracy: 0.9983 - loss: 0.0296 - val_accuracy: 0.9997 - val_loss: 0.0077
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step

ANN + GWO Results:
              precision    recall  f1-score   support

           0       0.05      0.43      0.09      4765
           1       1.00      0.99      1.00   7264104

    accuracy                           0.99   7268869
   macro avg       0.53      0.71      0.54   7268869
weighted avg       1.00      0.99      1.00   7268869

ROC-AUC: 0.7348400417551375


Applying particle swarm optimization over correlated feature dataset

PSO Feature Selection on 5 Chunks

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pyswarms as ps

# ---------------- CONFIG ----------------
CLEANED_DIR = "/content/drive/MyDrive/dataset/DDOS/processed_chunks"
CORR_FEATURES_PATH = "/content/drive/MyDrive/dataset/DDOS/corr_features.npy"
SAMPLE_CHUNKS = 5
SAVE_PSO_FEATURES_DIR = "/content/drive/MyDrive/dataset/DDOS/pso_features"
os.makedirs(SAVE_PSO_FEATURES_DIR, exist_ok=True)
# ---------------------------------------

# Load correlation-filtered features
filtered_features = np.load(CORR_FEATURES_PATH, allow_pickle=True)

files = sorted([f for f in os.listdir(CLEANED_DIR) if f.endswith(".parquet")])[:SAMPLE_CHUNKS]

def fitness_function(mask, X, y):
    """
    Fitness = 1 - accuracy (we want to minimize)
    mask: binary vector indicating selected features
    """
    mask = mask.astype(bool)
    if np.sum(mask) == 0:
        return 1.0  # penalize empty selection
    X_subset = X.iloc[:, mask]
    X_train, X_val, y_train, y_val = train_test_split(X_subset, y, test_size=0.2, random_state=42)
    clf = RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    return 1 - accuracy_score(y_val, y_pred)

for f in files:
    df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
    label_col = [c for c in df.columns if "label" in c.lower()][0]

    df = df[filtered_features + [label_col]]  # keep filtered features + label
    X_chunk = df[filtered_features]
    y_chunk = df[label_col]

    n_features = X_chunk.shape[1]

    # PSO: Binary optimization
    def objective_function(particles):
        n_particles = particles.shape[0]
        j = [fitness_function(particles[i], X_chunk, y_chunk) for i in range(n_particles)]
        return np.array(j)

    options = {'c1': 2, 'c2': 2, 'w': 0.9}
    optimizer = ps.discrete.BinaryPSO(n_particles=15, dimensions=n_features, options=options)
    best_cost, best_pos = optimizer.optimize(objective_function, iters=10)  # small iter for memory safety

    # Save selected features for this chunk
    selected_features = X_chunk.columns[best_pos.astype(bool)].tolist()
    chunk_name = f.replace(".parquet","")
    np.save(os.path.join(SAVE_PSO_FEATURES_DIR, f"{chunk_name}_pso.npy"), selected_features)
    print(f"Chunk {chunk_name}: Selected {len(selected_features)} features via PSO")




Combine All Chunks Using Filtered Features

In [ ]:
# ---------------- COMBINE CHUNKS ----------------
dfs = []
pso_feature_files = sorted([f for f in os.listdir(SAVE_PSO_FEATURES_DIR) if f.endswith("_pso.npy")])

for f, feat_file in zip(files, pso_feature_files):
    df = pd.read_parquet(os.path.join(CLEANED_DIR, f))
    chunk_features = np.load(os.path.join(SAVE_PSO_FEATURES_DIR, feat_file), allow_pickle=True)
    label_col = [c for c in df.columns if "label" in c.lower()][0]
    df = df[list(chunk_features) + [label_col]]
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
print("Combined shape:", combined_df.shape)

# Split features and labels
label_col = [c for c in combined_df.columns if "label" in c.lower()][0]
X_combined = combined_df.drop(columns=[label_col])
y_combined = combined_df[label_col]


PSO-Optimized ANN Training

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# ---------------- DATA SCALING ----------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_combined, test_size=0.2, random_state=42)

# ---------------- ANN MODEL ----------------
input_dim = X_train.shape[1]

model = Sequential([
    Dense(64, activation='relu', input_dim=input_dim),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ---------------- TRAIN ----------------
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=1024,
    epochs=3,
    verbose=1
)

# ---------------- EVALUATE ----------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {acc*100:.2f}%")
